# Deep lesson: async/await (Python)

Objectives:
- Understand the event loop, coroutine, task, and `await`.
- See clear runnable examples showing concurrency, timeouts, and integration with blocking code.
- Provide a tested async retry pattern and exercises for learners.

## Primer: event loop, coroutine, task, await

- A *coroutine* is a function defined with `async def` that can `await` other coroutines.
- The *event loop* runs coroutines and schedules I/O work.
- A *Task* wraps a coroutine and schedules it concurrently (use `asyncio.create_task`).
- Use `await` to suspend the current coroutine until the awaited coroutine completes.

In [4]:
import asyncio
import time

async def io_task(name, delay):
    """Simulated IO-bound work using asyncio.sleep.
    Returns the name after sleeping.
    """
    print(f'{name} start (delay={delay})')
    await asyncio.sleep(delay)
    print(f'{name} done')
    return name

async def sequential_run():
    start = time.perf_counter()
    results = []
    for i, d in enumerate([1.0, 0.5, 0.3], 1):
        results.append(await io_task(f'seq-{i}', d))
    elapsed = time.perf_counter() - start
    return 'sequential', results, elapsed

async def concurrent_run():
    start = time.perf_counter()
    tasks = [asyncio.create_task(io_task(f'con-{i}', d)) for i, d in enumerate([1.0, 0.5, 0.3], 1)]
    results = await asyncio.gather(*tasks)
    elapsed = time.perf_counter() - start
    return 'concurrent', results, elapsed

async def main():
    seq_name, seq_res, seq_time = await sequential_run()
    con_name, con_res, con_time = await concurrent_run()
    print('Summary:')
    print(f'{seq_name} results={seq_res} time={seq_time:.3f}s')
    print(f'{con_name} results={con_res} time={con_time:.3f}s')
    # Assert concurrency gives a smaller elapsed time than sum of delays (approx)
    assert con_time < seq_time, 'concurrent should be faster than sequential'
    return seq_time, con_time

# Run the demo when the cell is executed
seq_time, con_time = asyncio.run(main())
print(f'Assertion passed: concurrent {con_time:.3f}s < sequential {seq_time:.3f}s')

RuntimeError: asyncio.run() cannot be called from a running event loop

## Running blocking code inside async (run_in_executor / to_thread)

Sometimes you have CPU-bound or blocking library calls. Use `loop.run_in_executor` or `asyncio.to_thread` (3.9+) to run them without blocking the event loop.

In [ ]:
import asyncio
import time
from concurrent.futures import ThreadPoolExecutor

def blocking_io(duration):
    """Simulate blocking IO or CPU-bound work with time.sleep.
    This would block the event loop if called directly.
    """
    print(f'blocking start {duration}s')
    time.sleep(duration)
    print('blocking done')
    return duration

async def run_blocking_in_executor():
    loop = asyncio.get_running_loop()
    start = time.perf_counter()
    # run two blocking calls concurrently in the thread pool
    with ThreadPoolExecutor() as pool:
        f1 = loop.run_in_executor(pool, blocking_io, 1.0)
        f2 = loop.run_in_executor(pool, blocking_io, 0.6)
        res = await asyncio.gather(f1, f2)
    elapsed = time.perf_counter() - start
    print(f'results={res} elapsed={elapsed:.3f}s')
    assert elapsed < 1.7, 'executor should run them concurrently (approx)'
    return res, elapsed

# Run the executor demo
res, elapsed = asyncio.run(run_blocking_in_executor())
print('Executor demo completed')

## Cancellation and timeouts

Use `asyncio.wait_for` to enforce timeouts and handle `asyncio.TimeoutError`/`CancelledError`.

In [ ]:
import asyncio

async def long_task():
    print('long_task starting')
    await asyncio.sleep(2)
    print('long_task finished')
    return 'done'

async def timeout_demo():
    try:
        # this will timeout before long_task completes
        res = await asyncio.wait_for(long_task(), timeout=0.5)
    except asyncio.TimeoutError:
        print('timeout occurred as expected')
        return 'timed_out'
    else:
        return res

val = asyncio.run(timeout_demo())
assert val == 'timed_out', 'wait_for should have timed out'
print('Timeout demo behaved as expected')

## Exercise: async retry with exponential backoff

Implement an async retry helper that calls a flaky coroutine up to `max_attempts` times with exponential backoff (`base_delay * 2**attempt`). The helper should raise the last exception if all attempts fail.

In [ ]:
import asyncio
import random

class Flaky:
    def __init__(self, fail_times=2):
        self._calls = 0
        self.fail_times = fail_times

    async def maybe_fail(self):
        self._calls += 1
        print(f'flaky attempt {self._calls}')
        await asyncio.sleep(0.1)
        if self._calls <= self.fail_times:
            raise RuntimeError('transient error')
        return 'success'

async def async_retry(coro_func, max_attempts=4, base_delay=0.1):
    last_exc = None
    for attempt in range(1, max_attempts + 1):
        try:
            return await coro_func()
        except Exception as exc:
            last_exc = exc
            if attempt == max_attempts:
                break
            delay = base_delay * (2 ** (attempt - 1))
            print(f'attempt {attempt} failed; sleeping {delay:.2f}s before retry')
            await asyncio.sleep(delay)
    raise last_exc

async def demo_retry_success():
    f = Flaky(fail_times=2)
    res = await async_retry(f.maybe_fail, max_attempts=5, base_delay=0.05)
    print('retry result:', res)
    return res

val = asyncio.run(demo_retry_success())
assert val == 'success'
print('Retry demo passed')

## Summary & next steps

- Use `async/await` for IO-bound concurrency.
- Use `run_in_executor` / `to_thread` for blocking calls.
- Protect long-running coroutines with timeouts and support cancellation.

Next: try converting these examples into small scripts, add unit tests (pytest) around `async_retry`, and explore `aiohttp` for real HTTP examples.